In [2]:
import pandas as pd
import os

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
import requests
from loguru import logger

In [ ]:
try:
    
    # GitHub API endpoint to trigger a workflow
    url = "https://api.github.com/repos/RJUNCC/buckystyle_league_championship_series/actions/workflows/128475690/dispatches"
    headers = {
            'Accept': 'application/vnd.github+json',
            'Authorization': f'Bearer {os.getenv("GITHUB_TOKEN")}',
            'X-GitHub-Api-Version': '2022-11-28',
            'Content-Type': 'application/x-www-form-urlencoded',
        }

    # data = '{"ref":"topic-branch","inputs":{"name":"Mona the Octocat","home":"San Francisco, CA"}}'
    response = requests.post(url=url, headers=headers)
    if response.status_code == 200:
        logger.info('Successful')
    else:
        logger.error(f'{response.status_code}')
        logger.error('Unsuccessful')
except Exception as e:
    logger.error(str(e))

SyntaxError: f-string: unmatched '(' (3972974817.py, line 7)

In [6]:
pd.DataFrame(dict(response.json()).get("workflow_runs")).head()

,id,name,node_id,head_branch,head_sha,path,display_title,run_number,event,status,conclusion,workflow_id,check_suite_id,check_suite_node_id,url,html_url,pull_requests,created_at,updated_at,actor,run_attempt,referenced_workflows,run_started_at,triggering_actor,jobs_url,logs_url,check_suite_url,artifacts_url,cancel_url,rerun_url,previous_attempt_url,workflow_url,head_commit,repository,head_repository
0,14076003302,Daily Bot Task,WFR_kwLONMz0JM8AAAADRv7D5g,main,d5831980e0d411f832d6a2b7fba53bb7e5797fab,.github/workflows/main.yml,Daily Bot Task,214,schedule,completed,success,128475690,36229309405,CS_kwDONMz0JM8AAAAIb29j3Q,https://api.github.com/repos/RJUNCC/buckystyle...,https://github.com/RJUNCC/buckystyle_league_ch...,[],2025-03-26T05:12:48Z,2025-03-26T05:13:28Z,"{'login': 'RJUNCC', 'id': 110212318, 'node_id'...",1,[],2025-03-26T05:12:48Z,"{'login': 'RJUNCC', 'id': 110212318, 'node_id'...",https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,None,https://api.github.com/repos/RJUNCC/buckystyle...,{'id': 'd5831980e0d411f832d6a2b7fba53bb7e5797f...,"{'id': 885847076, 'node_id': 'R_kgDONMz0JA', '...","{'id': 885847076, 'node_id': 'R_kgDONMz0JA', '..."
1,14075073826,Daily Bot Task,WFR_kwLONMz0JM8AAAADRvCVIg,main,d5831980e0d411f832d6a2b7fba53bb7e5797fab,.github/workflows/main.yml,Daily Bot Task,213,workflow_dispatch,completed,success,128475690,36227188280,CS_kwDONMz0JM8AAAAIb08GOA,https://api.github.com/repos/RJUNCC/buckystyle...,https://github.com/RJUNCC/buckystyle_league_ch...,[],2025-03-26T03:47:47Z,2025-03-26T03:48:40Z,"{'login': 'RJUNCC', 'id': 110212318, 'node_id'...",1,[],2025-03-26T03:47:47Z,"{'login': 'RJUNCC', 'id': 110212318, 'node_id'...",https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,None,https://api.github.com/repos/RJUNCC/buckystyle...,{'id': 'd5831980e0d411f832d6a2b7fba53bb7e5797f...,"{'id': 885847076, 'node_id': 'R_kgDONMz0JA', '...","{'id': 885847076, 'node_id': 'R_kgDONMz0JA', '..."
2,14070794488,Daily Bot Task,WFR_kwLONMz0JM8AAAADRq9I-A,main,dc4d7774abc29d5935657ecaeb3dfa4fd12ee74e,.github/workflows/main.yml,Daily Bot Task,212,workflow_dispatch,completed,success,128475690,36217187253,CS_kwDONMz0JM8AAAAIbrZrtQ,https://api.github.com/repos/RJUNCC/buckystyle...,https://github.com/RJUNCC/buckystyle_league_ch...,[],2025-03-25T21:50:47Z,2025-03-25T21:51:30Z,"{'login': 'RJUNCC', 'id': 110212318, 'node_id'...",1,[],2025-03-25T21:50:47Z,"{'login': 'RJUNCC', 'id': 110212318, 'node_id'...",https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,https://api.github.com/repos/RJUNCC/buckystyle...,None,https://api.github.com/repos/RJUNCC/buckystyle...,{'id': 'dc4d7774abc29d5935657ecaeb3dfa4fd12ee7...,"{'id': 885847076, 'node_id': 'R_kgDONMz0JA', '...","{'id': 885847076, 'node_id': 'R_kgDONMz0JA', '..."
3,14052177528,Daily Bot Task,WFR_kwLONMz0JM8AAAADRZM2eA,main,dc4d7774abc29d5935657ecaeb3dfa4fd12ee74e,.github/workflows/main.yml,Daily Bot Task,211,schedule,completed,success,128475690,36167983897,CS_kwDONMz0JM8AAAAIa8ejGQ,https://api.github.com/repos/RJUNCC/buckystyle...,https://github.com/RJUNCC/buckystyle_league_ch...,[],2025-03-25T05:13:33Z,2025-03-25T05:14:21Z,"{'login': 'RJUNCC', 'id': 110212318, 'node_id'...",1,[],2025-03-25T05:13:33Z,"{'login': 'RJUNCC', 'id': 110212318, 'node_id'...",https://api.github.com/repos/RJUNCC/buckystyle...,https://api.